In [3]:
import numpy as np
import pandas as pd
import re
import math
from matplotlib import pyplot as plt
import glob

In [4]:
all_dx = np.load('../sourcedata/data/HBN/phenotype/parsed/dx_list.npy')
sub_list = np.load('../sourcedata/data/HBN/phenotype/parsed/sub_list.npy')
dx_array = np.load('../sourcedata/data/HBN/phenotype/parsed/dx_onehot.npy')
hbn_dir='/nese/mit/group/sig/projects/hbn/hbn_bids/'

In [5]:
dx_df = pd.DataFrame(dx_array)
dx_df.columns = all_dx

### list how many func files are there for different clinical groups

In [6]:
#get the ids of different dx
all_subs=dx_df.index
sub_ind=dx_df.loc[dx_df['No Diagnosis Given'] == 1].index
sub_ind_asd=dx_df.loc[dx_df['Autism Spectrum Disorder'] == 1].index
only_asd= dx_df.loc[((dx_df['ADHD-Combined Type'] == 0) & (dx_df['ADHD-Hyperactive/Impulsive Type'] == 0) & (dx_df['ADHD-Inattentive Type'] == 0) &  (dx_df['Unspecified Attention-Deficit/Hyperactivity Disorder'] == 0) & (dx_df['Autism Spectrum Disorder'] == 1) )].index

In [8]:
#function#give a list of subject indices and it will search for their functional files
#also give string of the clinical group
def count_funcs(list_in, group):
    hbn_dir='/nese/mit/group/sig/projects/hbn/hbn_bids/'
    sub_count=len(list_in)
    rest_DM_TP=0
    DM_TP=0
    rest=0
    anything=0
    peer=0
    for sub in sub_list[list_in]:
        #print(f)
        func_files=glob.glob(f'{hbn_dir}sub-{sub}/*/func/*')
        if (any("DM" in s for s in func_files) and any("TP" in s for s in func_files) and any("rest" in s for s in func_files)):
            rest_DM_TP+=1
        if (any("DM" in s for s in func_files) and any("TP" in s for s in func_files)):
            DM_TP +=1
        if (any("rest" in s for s in func_files)):
            rest+=1
        if (any("DM" in s for s in func_files) or any("TP" in s for s in func_files) or any("rest" in s for s in func_files)):
            anything+=1
        if (any("peer" in s for s in func_files)):
            peer+=1
    print(f'How many {group} subjects are there:')
    print(f'{sub_count} {group} subjects')
    print(f'{rest_DM_TP} saw both movies and had resting state')
    print(f'{DM_TP} saw both movies and no resting state')
    print(f'{rest} at least had resting state')
    print(f'{peer} had some peer data')
    print(f'{anything} at least had some functional data')

In [48]:
count_funcs(all_subs, 'all')

How many all subjects are there:
3615 all subjects
1453 saw both movies and had resting state
1616 saw both movies and no resting state
2211 at least had resting state
1896 had some peer data
2419 at least had some functional data


In [49]:
count_funcs(sub_ind, 'NT')

How many NT subjects are there:
308 NT subjects
115 saw both movies and had resting state
138 saw both movies and no resting state
200 at least had resting state
152 had some peer data
225 at least had some functional data


In [50]:
count_funcs(sub_ind_asd, 'ASC')

How many ASC subjects are there:
539 ASC subjects
215 saw both movies and had resting state
243 saw both movies and no resting state
315 at least had resting state
289 had some peer data
352 at least had some functional data


In [51]:
count_funcs(only_asd, 'ASC (no ADHD)')

How many ASC (no ADHD) subjects are there:
142 ASC (no ADHD) subjects
42 saw both movies and had resting state
51 saw both movies and no resting state
73 at least had resting state
68 had some peer data
85 at least had some functional data


### get list of ~50 NTs and ASCs to datalad get and fmriprep

In [17]:
NT_list=[]
for sub in sub_list[sub_ind]:
    func_files=glob.glob(f'{hbn_dir}sub-{sub}/*/func/*')
    if (any("DM" in s for s in func_files) and any("TP" in s for s in func_files) and any("rest" in s for s in func_files)):
        NT_list.append(f'sub-{sub}')
NT_list=NT_list[:50]

In [18]:
ASC_list=[]
for sub in sub_list[sub_ind_asd]:
    func_files=glob.glob(f'{hbn_dir}sub-{sub}/*/func/*')
    if (any("DM" in s for s in func_files) and any("TP" in s for s in func_files) and any("rest" in s for s in func_files)):
        ASC_list.append(f'sub-{sub}')
ASC_list=ASC_list[:50]

In [19]:
sub_list=NT_list
for i in ASC_list:
    sub_list.append(i)

In [23]:
n_sub_list = ["{}\n".format(i) for i in sub_list]
with open(r'hbn_100_subs.txt', 'w') as fp:
    fp.writelines(n_sub_list)